In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, MessagesState,START,END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal,TypedDict, List
from langchain_core.tools import tool

from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3)
    response = llm.invoke("Are you ready to help me")
    print(response.content)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    

Yes, I'm ready to help you.  Please tell me what you need.


## Document Load

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [4]:
file_path = 'data/cover_letter.pdf'
loader = PyPDFLoader(file_path,mode='single')
docs=loader.load()
spliter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0,separator="\n")
texts = spliter.split_documents(docs)

In [5]:
texts

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Cover letter for apexanalytix', 'source': 'data/cover_letter.pdf', 'total_pages': 2}, page_content='Natdanai  Intraraksa  \nAI  engineer  /  Data  Scientist   3217  w  Bertaue  Av.  Chicago,  IL  60618  \n Intraraksa@gmail.com  \n773-876-2897\n \n June  29,  2025  \nHiring  manager  \nApexanalytix  .Inc  \n I  am  writing  to  express  my  strong  interest  in  the  AI  Engineer  \nposition.\n \nWith\n \nover'),
 Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Cover letter for apexanalytix', 'source': 'data/cover_letter.pdf', 'total_pages': 2}, page_content='five\n \nyears\n \nof\n \nexperience\n \nin\n \nAI\n \nengineering\n \nand\n \ndata\n \nscience,\n \nalong\n \nwith\n \na\n \ndecade\n \nof\n \nexperience\n \nas\n \na\n \nmanufacturing\n \nengineer,\n \nI\n \nbring\n \na\n \nwell-rounded\n \nb

In [6]:
# docs=loader.load()

In [7]:
# docs

In [8]:
# spliter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0,separator="\n\n")
# texts = spliter.split_documents(docs)

In [9]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,is_separator_regex=True, separators=["\n\n", "\n", " ", ""])

In [10]:
# text_splitter.split_documents(docs)

## create vectorstore

In [11]:
# embedding 
embeddings = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')
vectorstore = Chroma.from_documents(texts,embedding=embeddings,collection_name="sample_collection",persist_directory="data/chroma_db")

## Create retrival tool

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(
    retriever=vectorstore.as_retriever(),
    name="retriever_tool",
    description="This tool is used to retrieve relevant documents from the vector store based on the user's query.")

In [ ]:
a = vectorstore.similarity_search("Which framework that I use to build the agent?")

[Document(id='92b3ae00-ae02-4033-aa00-d8ea990930d3', metadata={'creationdate': '', 'title': 'Cover letter for apexanalytix', 'creator': 'PyPDF', 'producer': 'Skia/PDF m140 Google Docs Renderer', 'source': 'data/cover_letter.pdf', 'total_pages': 2}, page_content='frameworks\n \nlike\n \nLangGraph\n \nand\n \nCrewAI\n \nto\n \nmanage\n \nautonomous\n \ntask\n \ndelegation\n \nand\n \nexecution\n \n \n I  have  hands-on  experience  fine-tuning  LLMs  with  techniques  such  as  \nLoRA,\n \nQLoRA,\n \nand\n \ninstruction\n \ntuning,\n \nand\n \nhave\n \nworked\n \nextensively\n \nwith\n \nLangChain,'),
 Document(id='cd34aa2d-aec5-446f-a441-8e782f43c915', metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'source': 'data/cover_letter.pdf', 'creationdate': '', 'total_pages': 2, 'title': 'Cover letter for apexanalytix', 'creator': 'PyPDF'}, page_content='frameworks\n \nlike\n \nLangGraph\n \nand\n \nCrewAI\n \nto\n \nmanage\n \nautonomous\n \ntask\n \ndelegation\n \nand\n \nexecutio

In [46]:
@tool
def retrieve_documents(query: str):
    """Tools for retrive information that relevant to my information of the users"""
    results = vectorstore.similarity_search(query)
    return results

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b + 9

tools = [retrieve_documents, multiply_numbers]

def query_vectorstore(state:MessagesState):
    prompt = ''' you are AI agent that answer the question by using tools'''
    agent = create_react_agent(llm, tools=tools, prompt=prompt)
    result = agent.invoke({'messages':state['messages']})
    return {'messages': result}

In [47]:
state = MessagesState()
state['messages'] = "Please describe  most recent role at Invitrace"

r = query_vectorstore(state)

In [48]:
r

{'messages': {'messages': [HumanMessage(content='Please describe  most recent role at Invitrace', additional_kwargs={}, response_metadata={}, id='355535b5-974d-4d69-a214-9e47f513b236'),
   AIMessage(content="I am sorry, I do not have access to private information about individuals' employment history, including roles at Invitrace.  My knowledge is based on publicly available information.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--a402ec47-bf2e-41a9-94be-1c9460f228b2-0', usage_metadata={'input_tokens': 54, 'output_tokens': 35, 'total_tokens': 89, 'input_token_details': {'cache_read': 0}})]}}

In [108]:
## Grade Node
def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        llm.bind_tools([retriver_tool]).invoke(state["messages"])
    )
    return {"messages": [response]}

In [100]:
def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """

    prompt = ''' Query relevant documents from the vector store based on the user's query.'''

    agent = create_react_agent(llm ,tools=[retriver_tool], prompt=prompt)
    response = agent.invoke({'messages':state["messages"]})
    return {"messages": [response]}

In [109]:
state = MessagesState()
state['messages'] = "Which framework that I use to build the agent?"

In [110]:
a = generate_query_or_respond(state)

In [113]:
a['messages'][0].pretty_print()

================================== Ai Message ==================================

The provided context only includes a `retriever_tool` function, which is a component that could be used within a larger agent framework.  There's no information about a complete agent framework.  To build an agent, you would need to choose a framework yourself, such as LangChain or similar.  The `retriever_tool` could then be integrated as a component within that framework.
